In [0]:
%sql
VACUUM default.parking_cleaned RETAIN 168 hours;

OPTIMIZE default.parking_cleaned;

In [0]:
from pyspark.sql.functions import to_date, hour, minute, date_format, to_timestamp, when, lit
from pyspark.sql.functions import col, max as max_
from pyspark.sql.types import IntegerType


# Hent 9 observasjoner per kjøring (én per lokasjon)

# Les inn parking_cleaned (silver) på nytt, slik at koden er uavhengig av df_cleaned i minne
df_cleaned = spark.table("default.parking_cleaned")

new_fakt = df_cleaned.select(
    to_date("timestamp").alias("dato"),
    date_format(to_timestamp("timestamp"), "HH:mm").alias("klokkeslett"),
    col("Sted"),
    col("Antall_ledige_plasser").cast(IntegerType()).alias("Antall_ledige_plasser"),
    col("timestamp")
)


gold_path = "/mnt/gold/fakt_parkering"

(
    new_fakt
    .write
    .format("delta")
    .mode("append")  # append for incremental loads
    .option("mergeSchema", "true")
    .save(gold_path)
)


# Register as Gold table in Hive Metastore
spark.sql(f"""
CREATE TABLE IF NOT EXISTS default.fakt_parkering
USING DELTA
LOCATION '{gold_path}'
""")



new_fakt.write.format("delta").mode("overwrite").saveAsTable("default.fakt_parkering_siste9")
